# Một số thông tin về bộ dữ liệu
- Nguồn gốc của bộ dữ liệu: [Mental Health Corpus](https://www.kaggle.com/datasets/reihanenamdari/mental-health-corpus).

### Mô tả bộ dữ liệu
- Bộ dữ liệu là dạng Corpus (1 tập hợp các văn bản) trong ngôn ngữ Tiếng Anh.
- Bộ Corpus gồm 2 cột `text` và `label` và 27977 hàng.
- Cột `text` biểu thị cho những bình luận đến từ những người dùng trên nền tảng mạng xã hội.
- Cột `label` gồm các giá trị 0 và 1, trong đó 1 biểu thị cho dấu hiệu của nội dung độc hại hoặc có liên kết với nguy cơ về các vấn đề tâm lý của người viết văn bản đó và 0 được coi là văn bản có thể bỏ qua trong mục đích trên.

***Ghi chú:*** Bộ dữ liệu được chia sẻ trên nền tảng [Kaggle](https://www.kaggle.com/) bởi [Reihaneh Namdari](https://www.kaggle.com/reihanenamdari), một nhà phân tích dữ liệu và nhà tâm lý trị liệu. Do những lo ngại về quyền riêng tư, có rất ít thông tin về bộ dữ liệu ngoài những thông tin được cung cấp tại đây. Điều này đưa ra một vài lưu ý:

- Chúng ta không biết dữ liệu đã được thu thập ở nền tảng cụ thể nào hay bằng cách nào.
- Chúng ta không biết thêm được thông tin gì về cá nhân đã viết ra các bình luận.
- Chúng ta không biết bộ dữ liệu đã được thu thập và tạo ra và khoảng thời gian nào.
- Chúng ta chưa thể chắc bộ dữ liệu này có thực sự 100% là dữ liệu từ đời thật.
- Chúng ta chưa thể chắc rằng bộ dữ liệu đã qua các bước xử lí nào và các bước đó đã được thực hiện trên toàn bộ bộ dữ liệu hay chưa.
- Chúng ta chưa thể chắc nếu dữ liệu đã qua các bước chọn lọc, hay thậm chí là chứa dữ liệu mô phỏng khi chưa qua phân tích.

In [174]:
# Nhập thư viện
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

# Đọc bộ dữ liệu
corpus = pd.read_csv('data/mental_health.csv')

# Tiền xử lý dữ liệu

- Trước khi đi vào phân tích và dọn dữ liệu, ta thực hiện một số bước xử lý tiêu chuẩn khi tiếp cận 1 Corpus. Phần tiền xử lý dữ liệu sẽ đảm bảo những quy trình sau được thuận lợi và giảm bớt trở ngại và tài nguyên xử lý.
- Mục đích của phần này là từ bộ Corpus gốc `./data/mental_health.csv`, ta thu được `./data/processed_mhc.csv` đã được xử lý sơ bộ (*mhc* viết tắt cho Mental Health Corpus).

### Các bước tiền xử lý dữ liệu

1. Loại bỏ dấu câu, các ký tự đặc biệt, chữ số.
2. Chuyển mọi ký tự chữ cái in hoa về in thường (Lowercasing).
3. Loại bỏ các đường liên kết phổ biển (Chứa *http*, *www*,...).
4. Chuẩn hóa các ký tự như *Tab* hay nhiều dấu cách về 1 dấu cách.
5. Loại bỏ các điểm trống hoặc chỉ có khoảng trắng.
6. Loại bỏ các điểm trùng nhau.

In [182]:
# Bước 1: Loại bỏ số, dấu câu, ký tự đặc biệt, chữ số
def check_for_special(text):
    return bool(re.search(r'[^a-zA-Z\s]', text))

rows_with_special = corpus[corpus['text'].apply(check_for_special)]
print(f"Total number of rows with numbers, punctuation, or special symbols: {rows_with_special.shape[0]}")

unique_chars_removed = set(re.findall(r'[^a-zA-Z\s]', ' '.join(corpus['text'].values)))
print(f"Unique characters removed: {unique_chars_removed}")

corpus['text'] = corpus['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

affected_rows = (rows_with_special.shape[0])  # since we are directly modifying the column
print(f"Number of rows affected: {affected_rows}")

remaining_special_rows = corpus[corpus['text'].apply(check_for_special)]
print(f"Number of rows with numbers, punctuation, or special symbols after cleaning: {remaining_special_rows.shape[0]}")

Total number of rows with numbers, punctuation, or special symbols: 0
Unique characters removed: set()
Number of rows affected: 0
Number of rows with numbers, punctuation, or special symbols after cleaning: 0


In [183]:
# Bước 2: Chuyển mọi ký tự chữ cái in hoa về in thường
def to_lowercase(text):
    return text.lower()

rows_with_uppercase = corpus[corpus['text'].str.contains(r'[A-Z]', regex=True)]
print(f"Total number of rows with uppercase letters before lowering case: {rows_with_uppercase.shape[0]}")

corpus['text'] = corpus['text'].apply(to_lowercase)

remaining_uppercase_rows = corpus[corpus['text'].str.contains(r'[A-Z]', regex=True)]
print(f"Total number of rows with uppercase letters after lowering case: {remaining_uppercase_rows.shape[0]}")

Total number of rows with uppercase letters before lowering case: 0
Total number of rows with uppercase letters after lowering case: 0


In [184]:
# Bước 3: Loại bỏ các liên kết
def contains_link(text):
    return bool(re.search(r'http|www', text))

rows_with_links_before = corpus[corpus['text'].apply(contains_link)]
print(f"Total number of rows with links before removal: {rows_with_links_before.shape[0]}")

corpus['text'] = corpus['text'].apply(lambda x: re.sub(r'http|www', '', x))

rows_with_links_after = corpus[corpus['text'].apply(contains_link)]
print(f"Total number of rows with links after removal: {rows_with_links_after.shape[0]}")

Total number of rows with links before removal: 0
Total number of rows with links after removal: 0


In [185]:
# Bước 4: Chuẩn hóa khoảng trắng
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

rows_with_extra_whitespace = corpus[corpus['text'].str.contains(r'\s{2,}|\t|\n', regex=True)]
print(f"Total number of rows with multiple spaces, tabs, or newlines: {rows_with_extra_whitespace.shape[0]}")

corpus['text'] = corpus['text'].apply(normalize_whitespace)

remaining_whitespace_rows = corpus[corpus['text'].str.contains(r'\s{2,}|\t|\n', regex=True)]
print(f"Number of rows with multiple spaces, tabs, or newlines after normalization: {remaining_whitespace_rows.shape[0]}")

Total number of rows with multiple spaces, tabs, or newlines: 0
Number of rows with multiple spaces, tabs, or newlines after normalization: 0


In [186]:
# Bước 5: Loại bỏ các điểm trống hoặc chỉ có khoảng trắng
total_rows_before = corpus.shape[0]
print(f"Total rows before removing empty or whitespace-only rows: {total_rows_before}")

corpus = corpus[corpus['text'].str.strip().astype(bool)]

total_rows_after = corpus.shape[0]
print(f"Total rows after removing empty or whitespace-only rows: {total_rows_after}")

rows_removed = total_rows_before - total_rows_after
print(f"Total rows removed: {rows_removed}")

Total rows before removing empty or whitespace-only rows: 27969
Total rows after removing empty or whitespace-only rows: 27969
Total rows removed: 0


In [187]:
# Bước 6: Loại bỏ các điểm trùng nhau
before_drop_dupes = corpus.shape[0]
print(f"Total rows before removing duplicates: {before_drop_dupes}")

duplicates = corpus[corpus.duplicated(subset=['text'], keep=False)]

print("Indices of duplicate rows:")
print(duplicates.index.tolist())
print("\nContent of duplicate rows:")
print(duplicates.head(len(corpus.duplicated(subset=['text'], keep=False))))

corpus = corpus.drop_duplicates(subset=['text'], keep='first')

print(f"\nTotal rows after removing duplicates: {corpus.shape[0]}")

rows_removed = before_drop_dupes - corpus.shape[0]
print(f"Number of rows removed: {rows_removed}")

remaining_duplicates = corpus.duplicated(subset=['text']).sum()
print(f"Number of duplicate rows after cleaning: {remaining_duplicates}")

Total rows before removing duplicates: 27969
Indices of duplicate rows:
[]

Content of duplicate rows:
Empty DataFrame
Columns: [text, label]
Index: []

Total rows after removing duplicates: 27969
Number of rows removed: 0
Number of duplicate rows after cleaning: 0


In [181]:
# Lưu lại `data/processed_mhc.csv`
corpus.to_csv('data/processed_mhc.csv', index=False)